<a href="https://colab.research.google.com/github/Malik-Raheel/Trading-Strategies/blob/main/Alpha_Bayesian_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bayesian model for 4 hour time frame

In [2]:
!pip install ccxt
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install joblib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11


In [4]:
import ccxt
import pandas as pd
import numpy as np
from sklearn.linear_model import BayesianRidge
from datetime import datetime, timedelta
import joblib

# Define global variables for prior data storage
MODEL_FILE = "bayesian_model.pkl"

def fetch_4h_data(api, asset, limit=200):
    """Fetch 4-hour OHLCV data from KuCoin."""
    try:
        ohlcv = api.fetch_ohlcv(asset, timeframe="4h", limit=limit)
        df = pd.DataFrame(ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        return df
    except Exception as e:
        print(f"Error fetching data for {asset}: {e}")
        return None

def preprocess_data(df):
    """Preprocess data to compute features."""
    # Calculate OBV
    df['obv'] = (np.sign(df['close'].diff()) * df['volume']).cumsum()
    df['obv_change'] = df['obv'].pct_change()

    # Calculate RSI
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=14).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
    rs = gain / loss
    df['rsi'] = 100 - (100 / (1 + rs))

    # Calculate ATR
    df['tr'] = np.maximum(df['high'] - df['low'],
                          np.maximum(abs(df['high'] - df['close'].shift()),
                                     abs(df['low'] - df['close'].shift())))
    df['atr'] = df['tr'].rolling(window=14).mean()

    # Drop rows with NaN values
    df = df.dropna()
    return df

def train_bayesian_model(X, y):
    """Train or update the Bayesian model."""
    # Load the model if it exists; otherwise, create a new one
    try:
        model = joblib.load(MODEL_FILE)
        print("Loaded existing Bayesian model.")
    except FileNotFoundError:
        model = BayesianRidge()
        print("Initialized new Bayesian model.")

    # Update the model with the new data
    model.fit(X, y)

    # Save the updated model
    joblib.dump(model, MODEL_FILE)
    return model

def predict_with_bayesian_model(model, X):
    """Make predictions with Bayesian model."""
    y_pred, y_std = model.predict(X, return_std=True)
    return y_pred, y_std

def fetch_signals_bayesian(api, assets):
    """Fetch signals using Bayesian regression for multiple assets."""
    results = []
    for asset in assets:
        df = fetch_4h_data(api, asset)
        if df is None:
            continue

        df = preprocess_data(df)
        X = df[["obv_change", "rsi", "atr"]]
        y = df["close"].shift(-1).dropna()
        X = X.iloc[:-1]

        # Train/update the model
        model = train_bayesian_model(X, y)

        # Predict the next close price
        latest_data = X.iloc[-1].values.reshape(1, -1)
        predicted_price, uncertainty = predict_with_bayesian_model(model, latest_data)

        # Get the latest actual close price
        latest_close = df['close'].iloc[-1]

        # Generate signal
        signal = "Buy" if predicted_price > latest_close else "Sell"

        results.append({
            "asset": asset,
            "signal": signal,
            "current_price": latest_close,
            "predicted_price": predicted_price[0],
            "uncertainty": uncertainty[0],
            "rsi": df['rsi'].iloc[-1],
            "atr": df['atr'].iloc[-1],
            "obv_change": df['obv_change'].iloc[-1]
        })
    return results

def display_bayesian_signals(signals):
    """Display Bayesian signals in a user-friendly format."""
    print("\n--- Bayesian Trading Signals ---")
    for signal in signals:
        print(f"Asset: {signal['asset']}")
        print(f"   Signal: {signal['signal']} (Predicted Price > Current Price: Buy)")
        print(f"   Current Price: ${signal['current_price']:.6f}")
        print(f"   Predicted Price: ${signal['predicted_price']:.6f} ± ${signal['uncertainty']:.6f}")
        print(f"   RSI: {signal['rsi']:.2f} (Above 50: Bullish; Below 50: Bearish)")
        print(f"   ATR: {signal['atr']:.5f} (Higher indicates more volatility)")
        print(f"   OBV Change: {signal['obv_change']:.4f} (Positive = Buying Pressure; Negative = Selling Pressure)")
        print()

# Initialize CCXT for KuCoin
exchange = ccxt.kucoin()

# Define the list of assets
assets = ["BTC/USDT", "ETH/USDT", "ADA/USDT", "NEAR/USDT",
          "XRP/USDT", "AVAX/USDT", "LINK/USDT", "CFX/USDT",
          "XAVA/USDT", "STG/USDT", "FET/USDT" , "PEPE/USDT" , "DOGE/USDT" , "SLIM/USDT" ,
          "GAFI/USDT" , "XCN/USDT" , "JUP/USDT" , "MNT/USDT" , "LTC/USDT" , "CRV/USDT" ,
          "SAND/USDT" , "PYTH/USDT" , "TAI/USDT"]

# Fetch Bayesian signals
signals = fetch_signals_bayesian(exchange, assets)

# Display the signals
display_bayesian_signals(signals)


Loaded existing Bayesian model.


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but BayesianRidge was fitted with feature names
  warnings.warn(


Loaded existing Bayesian model.


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but BayesianRidge was fitted with feature names
  warnings.warn(


Loaded existing Bayesian model.


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but BayesianRidge was fitted with feature names
  warnings.warn(


Loaded existing Bayesian model.
Loaded existing Bayesian model.


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but BayesianRidge was fitted with feature names
  warnings.warn(


Error fetching data for TAI/USDT: kucoin does not have market symbol TAI/USDT

--- Bayesian Trading Signals ---
Asset: BTC/USDT
   Signal: Sell (Predicted Price > Current Price: Buy)
   Current Price: $100270.300000
   Predicted Price: $92398.575315 ± $3429.980295
   RSI: 20.42 (Above 50: Bullish; Below 50: Bearish)
   ATR: 981.92857 (Higher indicates more volatility)
   OBV Change: -0.0227 (Positive = Buying Pressure; Negative = Selling Pressure)

Asset: ETH/USDT
   Signal: Buy (Predicted Price > Current Price: Buy)
   Current Price: $3159.970000
   Predicted Price: $3277.206842 ± $100.128415
   RSI: 29.16 (Above 50: Bullish; Below 50: Bearish)
   ATR: 46.14857 (Higher indicates more volatility)
   OBV Change: 0.0129 (Positive = Buying Pressure; Negative = Selling Pressure)

Asset: ADA/USDT
   Signal: Sell (Predicted Price > Current Price: Buy)
   Current Price: $0.911500
   Predicted Price: $0.884882 ± $0.058544
   RSI: 32.37 (Above 50: Bullish; Below 50: Bearish)
   ATR: 0.02114 (Hi

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but BayesianRidge was fitted with feature names
  warnings.warn(
